In [1]:
#!pip3 install http://download.pytorch.org/whl/cpu/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

!pip install fastai==1.0.61
!pip install torch==1.9.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 239 kB 890 kB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19191 sha256=b9b9788f355df71cad9b5b5eabdb11f98bcde844b5d94320123282f3baaf8a05
  Stored in directory: /root/.cache/pip/wheels/df/99/da/c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
Successfully built nvidia-ml-py3
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.0.1
    Uninstalling typing-extensions-4.0.1:
      Successfully uninstalled typing-extensions-4.0.1
  Attempting uninstall: fastai
    Found existing installation: fastai 2.5.3
    Uninstalling fastai-2.5.3:
      Successfully uninstalled fastai-2.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem

In [2]:
!pip install sentencepiece

In [3]:
from fastai.text import *
import numpy as np
#import demoji
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb
from sklearn.utils import shuffle
from nltk.corpus import words
from bs4 import BeautifulSoup
import nltk
nltk.download('words')
import nltk, string, re, spacy,unicodedata, random
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from sklearn.metrics import classification_report

[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.9.0+cu102')

In [5]:
torch.cuda.set_device(0)

In [6]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [7]:
random_seed(42, True)

In [8]:
# Train Data
df_train_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-train.csv', sep='\t', header=None)
df_train_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-train.csv', sep='\t', header=None)

# Dev Data
df_dev_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-dev.csv', sep='\t', header=None)
df_dev_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-dev.csv', sep='\t', header=None)

# Test Data
df_test_en = pd.read_csv('../input/tamilmisodatanew/ta-en-misogyny-test.csv', sep='\t', header=None)
df_test_ta = pd.read_csv('../input/tamilmisodatanew/ta-misogyny-test.csv', sep='\t', header=None)

In [9]:
path = Path('./')

In [10]:
!pwd

/kaggle/working


In [11]:
cat2ind = {'None-of-the-above': 0,
 'Transphobic': 1,
 'Counter-speech': 2,
 'Misandry': 3,
 'Homophobia': 4,
 'Hope-Speech': 5,
 'Xenophobia': 6,
 'Misogyny': 7,
 'Not-Tamil': 8}

ind2cat = dict()
for i in cat2ind:
    ind2cat[cat2ind[i]] = i

def restructure_df(df):
    txts = df[1].tolist()
    labs = df[0].tolist()
    labs_new = []
    for i in labs:
        labs_new.append(ind2cat[i])
    return pd.DataFrame({0:labs_new, 1:txts})

def over_under_sample(df, shuffle = True):
    def sample(rows, req_len):
        return [rows[i%len(rows)] for i in range(req_len)]

    if shuffle:
        df.sample(frac=1).reset_index(drop=True)
    data_dict = {'Misandry': [], 'Misogyny': [], 'Xenophobia': [], 'None-of-the-above': [], 'Hope-Speech': [], 'Counter-speech': [], 'Transphobic': [], 'Homophobia': [], 'Not-Tamil': []}

    for i in range(len(df)):
        data_dict[df[0][i]].append(list(df.iloc[i]))
    
    req_len = len(df)//len(data_dict)

    for lab in data_dict:
        data_dict[lab] = sample(data_dict[lab], req_len)
    df_rows = []
    for lab in data_dict:
        df_rows += data_dict[lab]
    df_new = pd.DataFrame(df_rows, columns = list(df.columns))
    if shuffle:
        df_new = df_new.sample(frac=1).reset_index(drop=True)
    return df_new

In [12]:
# df_train.shape, df_valid.shape, df_test.shape

In [13]:
df_train = df_train_en
df_valid = df_dev_en
df_test = df_test_en

In [14]:
df_train

,0,1
0,None-of-the-above,enaku unmaikum aluha wantu thirunangaigal thei...
1,Transphobic,SUPERSTAR VIJAY dai arivuketta polu ithu thapp...
2,None-of-the-above,Ugka smile cute a iruku😊😊...
3,None-of-the-above,Anna i am waiting na 🥰🥰🥰
4,None-of-the-above,Yanda tamilnadu la evvalavo pirachana iruku at...
...,...,...
5943,None-of-the-above,Noq day having no caste no religious certificate
5944,None-of-the-above,Mimicry escape aitaru bro. 😂
5945,None-of-the-above,Rajesh R u age u from
5946,Counter-speech,Bro unga videos nallarukku but don't do prank ...


In [15]:
df_train = shuffle(df_train, random_state = 10).reset_index(drop=True)
df_valid = shuffle(df_valid, random_state = 10).reset_index(drop=True)
df_test = shuffle(df_test, random_state = 10).reset_index(drop=True)

df_train[1] = df_train[1].apply(str)
df_valid[1] = df_valid[1].apply(str)
df_test[1] = df_test[1].apply(str)

In [16]:
def deEmojify(string):
    emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              u"\U00002500-\U00002BEF"  # chinese char
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  # dingbats
                              u"\u3030"
                              "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def preprocess(text):
    text = deEmojify(text) #convert emojis to their defns in words, they might be useful
    text = re.sub(r'([\.\'\"\/\-\_\--])',' ', text) # remove punctuations , removes @USER / some abbreviatins
    to_remove_url = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
      '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = re.sub(to_remove_url,'',text)  # remove url patterns
    text = re.sub(" \d+", " ", text)
    text = text.replace(","," ")
    text = re.sub(r'(?:^| )\w(?:$| )', ' ', text).strip()
    punctuation='!!"$%&()*+-/:;<=>?[\\]^_{|}~.'
    text = ''.join(ch for ch in text if ch not in set(punctuation))
    # text = text.translate(str.maketrans('', '', string.punctuation))
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Stopword Removing
    tokenizer = ToktokTokenizer()
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    text = ' '.join(ch for ch in tokens)
    return text 

def clean(df):
    df[1] = df[1].apply(lambda x: preprocess(x))

clean(df_train)
clean(df_valid)
clean(df_test)

In [17]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [18]:
class TamilTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('../input/spm-sota-models/tamil_spm_8k.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [19]:
sp = spm.SentencePieceProcessor()
sp.Load(str('../input/spm-sota-models/tamil_spm_8k.model'))
itos = [sp.IdToPiece(int(i)) for i in range(8000)]

In [20]:
tamil_vocab = Vocab(itos)

In [21]:
tokenizer = Tokenizer(tok_func=TamilTokenizer, lang='ta')

In [22]:
label_cols = [0]

In [23]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=tamil_vocab)

/opt/conda/lib/python3.7/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


In [24]:
data_lm.show_batch()

idx,text
0,ay a ▁ k il ic hi ▁ an up ir en ▁ un ▁p on d att i ▁ y a ▁ an up p i <unk> <unk> ai ▁the vi d iya ▁p ay al e ▁ x x b os <unk> <unk> th u ▁p ran k ▁ ill a ▁ en na iya ▁a al u h ava ik k um ▁ sh or t ▁f
1,ud a ▁ e an ak u ▁m att um th an ▁ app udi y on u ▁n en ach an ▁ x x b os <unk> d th u van th i ▁ y g ▁ ev ang al al a ▁a van ga ▁ com m un it ti k ke y ▁ ava man am ▁ x x b os <unk> <unk> ay ▁ na ▁
2,y ath a vi ra ▁ y ell am ▁p es uth u ▁ lo os u ▁ lo os u ▁m uth ▁ x x re p ▁4 ▁ i ▁p oc hi ▁ k ov am ▁ th an ▁ v ar uth u ▁ x x b os <unk> ▁a van ga ▁ iya l p av e <unk> <unk> om p a ▁ az ha ha
3,va ay an ▁m al ay al i ▁ van d her i <unk> ▁s im on ai y um ▁ u z h z ha ▁ th al li ▁n ung ed uk k an um ▁ x x b os <unk> <unk> n ▁ k an av ar ▁ th im ir u ▁ e ch u ▁p ud uc ha a ▁ all l ▁ x x
4,uk um ▁ ent ha ▁ th ap a ▁ se iya th a ▁ av ang al uk u ▁n ara ga ma ▁ il a ▁s org am an u ▁ j es us ▁ ka e ▁ ter ium ▁ com ment ▁p od a ▁ van ta n ung a ▁ x x b os ▁in th a ▁ g op in ath ▁ v en


In [25]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [26]:
learn.load('/kaggle/input/pretrained-sota-models/wikitalm_8k_447_third')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (5948 items)
x: LMTextList
▁ x x b os <unk> <unk> ▁ vi r un th al ik k u ▁p or ant ha ▁the vi d iya ▁p ay al e ▁ un na ▁ ung om ma ▁ ev ank u u ▁p ad uth u ▁ pe th u ▁ un na ▁ ung om ma ▁ ev ank u u ▁p ad uth u <unk> ▁a va ▁p un d ay a ▁ k il ic hi ▁ an up ir en ▁ un ▁p on d att i ▁ y a ▁ an up p i <unk> <unk> ai ▁the vi d iya ▁p ay al e,▁ x x b os <unk> <unk> th u ▁p ran k ▁ ill a ▁ en na iya ▁a al u h ava ik k um ▁ sh or t ▁f li m,▁ x x b os <unk> ▁n ee ▁m att um ▁m ad h am ▁ vi t tu ▁m ad h am ▁ ma ar u va ▁a van ga ▁ ma ara ▁ k ood ad ha,▁ x x b os <unk> <unk> o o d <unk> <unk> a m i l <unk> <unk> a n n e l ▁ d han as e k ar ▁p on n ung a ▁p un d ai ▁s up er ▁p un d a is um ma a ▁ g um m un u ▁ y er ud h u ▁ y en <unk> ▁s un ni,▁ x x b os <unk> ▁s a ad hi ▁ th a ▁ per us un u ▁p es ur a ▁ na a ing al a ▁ <unk> el l am ▁ se ru pp a ▁ ka la ti ▁ adi k an um
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (1488 items)
x: LMTextLis

In [27]:
df_train = df_train.drop([0]).reset_index().drop(["index"], axis=1)
df_train[0].value_counts()

None-of-the-above    3720
Misandry              829
Counter-speech        348
Xenophobia            297
Hope-Speech           213
Misogyny              211
Homophobia            172
Transphobic           157
Name: 0, dtype: int64

In [28]:
df_valid[0].value_counts()

None-of-the-above    919
Misandry             218
Counter-speech        95
Xenophobia            70
Hope-Speech           53
Misogyny              50
Homophobia            43
Transphobic           40
Name: 0, dtype: int64

In [29]:
df_test[0].value_counts()

None-of-the-above    1143
Misandry              292
Xenophobia             95
Counter-speech         88
Hope-Speech            70
Transphobic            58
Misogyny               57
Homophobia             56
Name: 0, dtype: int64

In [30]:
learn.freeze()

In [31]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.529577,3.267153,0.309834,00:08


In [32]:
learn.unfreeze()

In [33]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.218440,3.038661,0.348958,00:10
1,3.008652,2.826774,0.380518,00:10
2,2.840297,2.731074,0.395126,00:10
3,2.738980,2.693304,0.400632,00:10
4,2.693410,2.687332,0.402282,00:10


In [34]:
learn.predict('▁தனது ▁சொந்த',n_words=10)

'▁தனது ▁சொந்த 9 , ▁அ க்குள் , ▁நியூட்டன ் , ▁மற்றும் ▁நி'

In [35]:
learn.save_encoder('fine_tuned_enc')

In [36]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=tamil_vocab, label_cols=label_cols, bs=64)

/opt/conda/lib/python3.7/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [37]:
data_clas.show_batch()

text,target
▁ x x b os <unk> ▁s w ath i <unk> ▁s ank ar ra j ▁c ore ect ▁ ip o ▁ ne en ga le y ▁ o th uk ut ing a ▁p ud h u ▁ ed ath uk u ▁ po na ▁ ka ka ▁b ay am a ▁ er uk um ▁n u ▁ ad he y ▁ min d s et ▁,Counter-speech
▁ x x b os ▁ k att up a adu ▁p ir ak ra d h ul a ▁ en na ▁ th app u ▁ ir uk u ▁ ung la ▁ or u ▁ k att up a at uk ul la ▁ o z h uk am a ▁ v al ar ka ad ha d h u ▁ th a ▁ ip a ▁ ne,Homophobia
▁ x x b os ▁ <unk> man as at ch i ▁ or uk k ur a ▁ man ith ar ▁ th an ▁ ch ro mo s ome ▁p ro bl em s ▁ la a ▁p ill a in per ur ang al ▁ ath uk k u ▁ ar ov u ▁ ir uk ka ▁a ga in ▁ ra h ▁s ol lu ra,Counter-speech
▁ x x b os <unk> <unk> har an <unk> <unk> am il and a ▁ y and a <unk> ▁s om b u ▁ e th u ik k u ▁m un n adi <unk> <unk> ay al al ith a <unk> <unk> ar un an ith i ▁ se th app a ▁ y ell a ▁c han n el ▁p ot tu ▁ ka att un a,Counter-speech
▁ x x b os <unk> li ni ▁ pl s ▁ ne ▁ per a ▁m ath ik o ▁the v y a ▁n u ▁ 53 ▁ he y ▁n ay e ▁ un ak u ▁ av lo ▁ akk ara ▁ ir un th a ▁ <unk> an ad ha <unk> ▁a as ar am am ▁ ve ch i ▁ k ol and ha ing,Misogyny


In [38]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [39]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [40]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (5947 items)
x: TextList
▁ x x b os <unk> <unk> th u ▁p ran k ▁ ill a ▁ en na iya ▁a al u h ava ik k um ▁ sh or t ▁f li m,▁ x x b os <unk> ▁n ee ▁m att um ▁m ad h am ▁ vi t tu ▁m ad h am ▁ ma ar u va ▁a van ga ▁ ma ara ▁ k ood ad ha,▁ x x b os <unk> <unk> o o d <unk> <unk> a m i l <unk> <unk> a n n e l ▁ d han as e k ar ▁p on n ung a ▁p un d ai ▁s up er ▁p un d a is um ma a ▁ g um m un u ▁ y er ud h u ▁ y en <unk> ▁s un ni,▁ x x b os <unk> ▁s a ad hi ▁ th a ▁ per us un u ▁p es ur a ▁ na a ing al a ▁ <unk> el l am ▁ se ru pp a ▁ ka la ti ▁ adi k an um,▁ x x b os <unk> <unk> n ak u ▁ ung al a ▁ ro m b a ▁p id ik um ▁ an na ▁b ut ▁n u ▁the ri y ath u ▁ am ▁ al s o ▁s ta ff ▁n ur se
y: CategoryList
None-of-the-above,None-of-the-above,Misogyny,None-of-the-above,None-of-the-above
Path: .;

Valid: LabelList (1488 items)
x: TextList
▁ x x b os ▁ ta mi lan ▁ en ga ▁p on al um ▁m un er a ▁m ad ing al ▁ ip p adi y e ▁p es i ▁p es i ▁ ta m il ▁m

In [41]:
learn.freeze()

In [42]:
learn.loss_func.func

CrossEntropyLoss()

In [43]:
mcc = MatthewsCorreff()

In [44]:
learn.metrics = [mcc, accuracy]

In [45]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,1.383821,1.111634,0.304831,0.666667,00:06


/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


In [46]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,1.132208,1.003623,0.404080,0.699597,00:06


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

In [47]:
learn.unfreeze()
learn.fit_one_cycle(8, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.983383,0.976530,0.416414,0.705645,00:14
1,0.998329,0.946730,0.454905,0.714382,00:14
2,0.949423,0.895451,0.485285,0.729167,00:14
3,0.892631,0.901028,0.495061,0.731183,00:13
4,0.845476,0.884043,0.502734,0.731855,00:13
5,0.783543,0.892045,0.482495,0.723790,00:13
6,0.737612,0.905518,0.499313,0.723118,00:13
7,0.717013,0.896348,0.492454,0.723118,00:14


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 0 with accuracy value: 0.7056451439857483.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 1 with accuracy value: 0.7143816947937012.


...

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 3 with accuracy value: 0.7311828136444092.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

Better model found at epoch 4 with accuracy value: 0.7318548560142517.


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe

In [48]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]

i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key

df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


,query,actual_label,predicted_label,Homophobia,None-of-the-above,Misogyny,Transphobic,Hope-Speech,Misandry,Xenophobia,Counter-speech
0,Good percent correct allaga sunnega sir eppadi...,None-of-the-above,None-of-the-above,0.047014,0.658602,0.019722,0.01421,0.008899,0.038531,0.000861,0.21216
1,Sariyaga than solli irukkar virunthalikku pora...,Misandry,None-of-the-above,0.022858,0.670576,0.010712,0.026238,0.00595,0.142176,0.094326,0.027164
2,Kasi Ravi hey in velaiya paaru daa nedan da po...,Misandry,Misandry,0.006452,0.215612,0.036928,0.007829,0.001715,0.720401,0.009399,0.001665
3,raja vai adakka vendum,Misandry,Misandry,0.010959,0.345798,0.01227,0.007738,0.005284,0.44418,0.167828,0.005943
4,Chutti payyan sam very good morning,None-of-the-above,None-of-the-above,0.011098,0.916322,0.003903,0.003345,0.00443,0.04223,0.012966,0.005705


In [49]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.7105970952124798

In [50]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.46559313105691985

In [51]:
report = classification_report(df_result['actual_label'], df_result['predicted_label'],output_dict=True)

In [52]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
Counter-speech,0.388889,0.318182,0.350000,88.000000
Homophobia,0.520000,0.232143,0.320988,56.000000
Hope-Speech,0.327273,0.257143,0.288000,70.000000
Misandry,0.664260,0.630137,0.646749,292.000000
Misogyny,0.266667,0.070175,0.111111,57.000000
None-of-the-above,0.775758,0.895888,0.831506,1143.000000
Transphobic,0.363636,0.068966,0.115942,58.000000
Xenophobia,0.547619,0.484211,0.513966,95.000000
accuracy,0.710597,0.710597,0.710597,0.710597
macro avg,0.481763,0.369606,0.397283,1859.000000


In [53]:
df_result.to_csv('ULMFiT_ACD_TaEn_CD.csv', index=False)

In [54]:
import os
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'ULMFiT_ACD_TaEn_CD.csv')

/kaggle/working/ULMFiT_ACD_TaEn_CD.csv